In [2]:
#@title Imports

import collections
import copy
import itertools
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import loggamma
from scipy.stats import beta
import time

mpl.style.use("classic")
mpl.rcParams["figure.figsize"] = [5, 3]

mpl.rcParams["axes.linewidth"] = 0.75
mpl.rcParams["figure.facecolor"] = "w"
mpl.rcParams["grid.linewidth"] = 0.75
mpl.rcParams["lines.linewidth"] = 0.75
mpl.rcParams["patch.linewidth"] = 0.75
mpl.rcParams["xtick.major.size"] = 3
mpl.rcParams["ytick.major.size"] = 3

mpl.rcParams["pdf.fonttype"] = 42
mpl.rcParams["ps.fonttype"] = 42
mpl.rcParams["font.size"] = 12
mpl.rcParams["axes.titlesize"] = "medium"
mpl.rcParams["legend.fontsize"] = "medium"

import platform
print("python %s" % platform.python_version())
print("matplotlib %s" % mpl.__version__)

import tensorflow.compat.v2 as tf
# import tensorflow_probability as tfp
tf.enable_v2_behavior()
from tensorflow import keras

def linestyle2dashes(style):
  if style == "--":
    return (3, 3)
  elif style == ":":
    return (0.5, 2.5)
  else:
    return (None, None)

python 3.7.12
matplotlib 3.2.2


In [3]:
#@title Bandit simulator and environments

class GaussBandit(object):
  """Gaussian bandit."""

  def __init__(self, mu, sigma=0.5):
    self.mu = np.copy(mu)
    self.K = self.mu.size
    self.sigma = sigma

    self.best_arm = np.argmax(self.mu)

    self.randomize()

  def randomize(self):
    # generate random rewards
    self.rt = self.mu + self.sigma * np.random.randn(self.K)

  def reward(self, arm):
    # instantaneous reward of the arm
    return self.rt[arm]

  def regret(self, arm):
    # instantaneous regret of the arm
    return self.rt[self.best_arm] - self.rt[arm]

  def pregret(self, arm):
    # expected regret of the arm
    return self.mu[self.best_arm] - self.mu[arm]

  def print(self):
    return "Gaussian bandit with arms (%s)" % \
      ", ".join("%.3f" % s for s in self.mu)


class LinBandit(object):
  """Linear bandit."""

  def __init__(self, X, theta, noise="normal", sigma=0.5):
    self.X = np.copy(X)
    self.K = self.X.shape[0]
    self.d = self.X.shape[1]
    self.theta = np.copy(theta)
    self.noise = noise
    if self.noise == "normal":
      self.sigma = sigma

    self.mu = self.X.dot(self.theta)
    self.best_arm = np.argmax(self.mu)

    self.randomize()

  def randomize(self):
    # generate random rewards
    if self.noise == "normal":
      self.rt = self.mu + self.sigma * np.random.randn(self.K)
    elif self.noise == "bernoulli":
      self.rt = (np.random.rand(self.K) < self.mu).astype(float)
    elif self.noise == "beta":
      self.rt = np.random.beta(4 * self.mu, 4 * (1 - self.mu))

  def reward(self, arm):
    # instantaneous reward of the arm
    return self.rt[arm]

  def regret(self, arm):
    # instantaneous regret of the arm
    return self.rt[self.best_arm] - self.rt[arm]

  def pregret(self, arm):
    # expected regret of the arm
    return self.mu[self.best_arm] - self.mu[arm]

  def print(self):
    if self.noise == "normal":
      return "Linear bandit: %d dimensions, %d arms" % \
        (self.d, self.K)
    elif self.noise == "bernoulli":
      return "Bernoulli linear bandit: %d dimensions, %d arms" % \
        (self.d, self.K)
    elif self.noise == "beta":
      return "Beta linear bandit: %d dimensions, %d arms" % \
        (self.d, self.K)


In [12]:
#@title Baseline algorithms

class LinBanditAlg(object):
  def __init__(self, env, n, params):
    self.env = env
    self.n = n
    self.K = self.env.X.shape[0]
    self.d = self.env.X.shape[1]
    self.theta0 = np.zeros(self.d)
    self.sigma0 = 1.0
    self.sigma = 0.5
    self.crs = 1.0 # confidence region scaling

    for attr, val in params.items():
      setattr(self, attr, val)

    if not hasattr(self, "Sigma0"):
      self.Sigma0 = np.square(self.sigma0) * np.eye(self.d)

    # sufficient statistics
    self.Gram = np.linalg.inv(self.Sigma0)
    self.B = self.Gram.dot(self.theta0)

  def update(self, t, arm, r):
    x = self.env.X[arm, :]
    self.Gram += np.outer(x, x) / np.square(self.sigma)
    self.B += x * r / np.square(self.sigma)


class LinTS(LinBanditAlg):
  def get_arm(self, t):
    Gram_inv = np.linalg.inv(self.Gram)
    thetabar = Gram_inv.dot(self.B)

    # posterior sampling
    thetatilde = np.random.multivariate_normal(thetabar,
      np.square(self.crs) * Gram_inv, tol=1e-6)
    self.mu = self.env.X.dot(thetatilde)

    arm = np.argmax(self.mu)
    return arm

  @staticmethod
  def print():
    return "LinTS"

In [19]:
class LinExp4_Corall(object):
  """Exp4 with linear reward models where models are updated."""

  def __init__(self, env, n, params):
    self.env = env
    self.n = n
    self.K = self.env.X.shape[0]
    self.d = self.env.X.shape[1]
    self.sigma = 0.5
    self.crs = 1.0 # confidence region scaling

    # default prior parameters
    self.gamma = None
    self.eta = None
    self.n_latent = 1
    self.thetas0 = np.zeros((self.n_latent, self.d))
    self.sigmas0 = 0.1

    for attr, val in params.items():
      setattr(self, attr, val)

    # parse loaded prior parameters if passed.
    if not self.gamma:
      self.gamma = min(
          1, np.sqrt(self.K * np.log(self.K) / ((np.e - 1) * self.n)))
    if not self.eta:
      self.eta = self.gamma / self.K
    assert self.thetas0.shape == (self.n_latent, self.d)
    if isinstance(self.sigmas0, float):  # specified isotropic prior.
      self.sigmas0 = np.tile(
          self.sigmas0 * np.eye(self.d)[np.newaxis, ...], (self.n_latent, 1, 1))
    else:
      assert self.sigmas0.shape == (self.n_latent, self.d, self.d)

    self.log_w = np.ones(self.n_latent)

    # sufficient statistics
    self.Gram = np.zeros((self.d, self.d))
    self.B = np.zeros((self.d,))

    self.thetas = np.copy(self.thetas0)

  def update(self, t, arm, r):
    del t
    x = self.env.X[arm, :]
    self.Gram += np.outer(x, x) / np.square(self.sigma)
    self.B += x * r / np.square(self.sigma)

    # importance weighting to get counterfactual rewards.
    rewards = r / self.phat[arm] * np.eye(self.K)[arm]
    arms = np.argmax(self.thetas.dot(self.env.X.T), axis=-1)    # expert predictions.
    self.log_w += self.eta * rewards[arms]

    self._update_experts()

  def _update_experts(self):
    prec = self.Gram / self.sigma**2
    thetahat = np.dot(np.linalg.pinv(self.Gram), self.B)  # linear regression

    # compute conditional posterior for each latent state.
    prec0 = np.linalg.pinv(self.sigmas0)
    Sigmas = np.linalg.pinv(prec0 + prec[None, ...])
    prec_thetas0 = np.einsum('ijk,ik->ij', prec0, self.thetas0)
    thetabars = np.einsum(
        'ijk,ik->ij', Sigmas, prec_thetas0 + prec.dot(thetahat)[None, ...])
  
  def get_arm(self, t):
    del t
    # Compute mixture probabilities.
    arms = np.argmax(self.thetas.dot(self.env.X.T), axis=-1)
    w = np.exp(self.log_w - self.log_w.max())
    w /= w.sum()

    p =  np.dot(w, np.eye(self.K)[arms])
    # Action probabilities.
    self.phat = (1 - self.gamma) * p  + self.gamma / self.K

    # Pull arm stochastically.
    arm = np.random.choice(np.arange(self.phat.size), p=self.phat)
    return arm

  @staticmethod
  def print():
    return "LinExp4"

In [20]:
class MixtureLinTS(object):
  """Gaussian TS for linear bandits with mixture prior over parameters."""

  def __init__(self, env, n, params):
    self.env = env
    self.n = n
    self.K = self.env.X.shape[0]
    self.d = self.env.X.shape[1]
    self.sigma = 0.5
    self.crs = 1.0 # confidence region scaling

    # default prior parameters
    self.n_latent = 1
    self.thetas0 = np.zeros((self.n_latent, self.d))
    self.sigmas0 = 0.1
    self.init_p = None

    for attr, val in params.items():
      setattr(self, attr, val)

    # parse loaded prior parameters if passed.
    assert self.thetas0.shape == (self.n_latent, self.d)
    if isinstance(self.sigmas0, float):  # specified isotropic prior.
      self.sigmas0 = np.tile(
          self.sigmas0 * np.eye(self.d)[np.newaxis, ...], (self.n_latent, 1, 1))
    else:
      assert self.sigmas0.shape == (self.n_latent, self.d, self.d)
    if self.init_p is None:
      self.init_p = np.ones((self.n_latent,)) / self.n_latent
    else:
      assert self.init_p.shape == (self.n_latent,)

    # sufficient statistics
    self.Gram = np.zeros((self.d, self.d))
    self.B = np.zeros((self.d,))

    self.loglikes = np.zeros((self.n_latent))  # log-likelihoods
    # posterior parameters
    self.thetabars = np.copy(self.thetas0)
    self.Sigmas = np.copy(self.sigmas0)

  def update(self, t, arm, r):
    x = self.env.X[arm, :]
    self.Gram += np.outer(x, x) / np.square(self.sigma)
    self.B += x * r / np.square(self.sigma)

    ds = r - self.thetabars.dot(x)
    crs = np.dot(x.dot(self.Sigmas), x)
    self.loglikes -= 0.5 * np.square(ds) / (crs + self.sigma**2)

    self._update_posterior_params()

  def _update_posterior_params(self):
    prec = self.Gram / self.sigma**2
    thetahat = np.dot(np.linalg.pinv(self.Gram), self.B)  # linear regression

    # compute conditional posterior for each latent state.
    prec0 = np.linalg.pinv(self.sigmas0)
    self.Sigmas = np.linalg.pinv(prec0 + prec[None, ...])
    prec_thetas0 = np.einsum('ijk,ik->ij', prec0, self.thetas0)
    self.thetabars = np.einsum(
        'ijk,ik->ij', self.Sigmas, prec_thetas0 + prec.dot(thetahat)[None, ...])

  def get_arm(self, t):
    # posterior sampling
    p = self.init_p * np.exp(self.loglikes - self.loglikes.max())
    p /= p.sum()
    z = np.random.choice(np.arange(p.size), p=p)

    thetatilde = np.random.multivariate_normal(
        self.thetabars[z], self.Sigmas[z])
    self.mu = self.env.X.dot(thetatilde)

    arm = np.argmax(self.mu)
    return arm

  @staticmethod
  def print():
    return "MixtureLinTS"

In [ ]:
# linear bandit
alg_specs = [
    ("UniTS", "blue", "-"),
    ("MixTS", "green", "-")]
num_runs = 100
n = 1000

step = np.arange(1, n + 1)
sube = (step.size // 10) * np.arange(1, 11) - 1
d = 30
X = np.eye(d)

for L in [5, 10, 15, 30]:
  for sigma0 in [0.05]:
    # reward noise
    sigma = 0.1

    plt.figure(figsize=(4, 2.8))

    for alg_spec in alg_specs:
      regret = np.zeros((n, num_runs))

      for run in range(num_runs):
        # true hyper-prior
        s = np.random.randint(L)
        thetas0 = 0.1 * np.ones((L, d))
        thetas0[np.arange(L), np.arange(L)] = 0.9
        theta_star = thetas0[s] + sigma0 * np.random.randn(d)
        env = LinBandit(X, theta_star, sigma=sigma)

        # initialize algorithms
        if alg_spec[0] == 'UniTS':
          # TS
          alg_params = {
            "theta0": thetas0.mean(axis=0),
            "sigma": sigma}
          alg = LinTS(env, n, alg_params)
        elif alg_spec[0] == 'MixTS':
          # MixTS
          alg_params = {
            "n_latent": L,
            "thetas0": thetas0,
            "sigmas0": sigma0,
            "sigma": sigma}
          alg = MixtureLinTS(env, n, alg_params)

        for t in range(n):
          arm = alg.get_arm(t)
          r = env.reward(arm)
          alg.update(t, arm, r)
          regret[t, run] = env.regret(arm)

      cum_regret = regret.cumsum(axis=0)
      plt.plot(step, cum_regret.mean(axis=1),
        dashes=linestyle2dashes(alg_spec[2]), color=alg_spec[1],
        label=alg_spec[0])
      plt.errorbar(step[sube], cum_regret[sube, :].mean(axis=1),
        cum_regret[sube, :].std(axis=1) / np.sqrt(cum_regret.shape[1]),
        fmt="none", ecolor=alg_spec[1])

      print("%s: %.1f +/- %.1f" % (alg_spec[0],
        cum_regret[-1, :].mean(),
        cum_regret[-1, :].std() / np.sqrt(cum_regret.shape[1])))

    plt.title(r"Linear Bandit (L = %d, $\sigma_0$ = %.2f)" % (L, sigma0))
    plt.xlabel("Round t")
    plt.xticks(np.arange(n + 1, step=200))
    plt.ylabel("Regret")
    plt.ylim(bottom=0)
    plt.legend(loc="upper left", frameon=False, prop={'size': 10})

    plt.tight_layout()
    plt.show()